In [ ]:
from cnn.model import build_net
import tensorflow as tf

In [ ]:
layer_list = [
    'vgg_d_3',
    'vgg_n_3',
    'vgg_n_3',
    'vgg_d_3',
    'vgg_n_3',
    'vgg_n_3',
    'vgg_u_3',
    'vgg_n_3',
    'vgg_u_3',
    'vgg_n_3',
]

# cell_list = [
#     "NonscalingCell",
#     "DownscalingCell",
#     "DownscalingCell",
#     "DownscalingCell",
#     "DownscalingCell",
#     "DownscalingCell",
#     "DownscalingCell",
#     "DownscalingCell",
#     "DownscalingCell",
#     "NonscalingCell",
# ]

fn_dict =  {
    'den_d_3': {'cell': 'DownscalingCell', 'block': 'DenseBlock',     'kernel': 3},
    'den_d_5': {'cell': 'DownscalingCell', 'block': 'DenseBlock',     'kernel': 5},
    'den_d_7': {'cell': 'DownscalingCell', 'block': 'DenseBlock',     'kernel': 7},
    'den_n_3': {'cell': 'NonscalingCell',  'block': 'DenseBlock',     'kernel': 3},
    'den_n_5': {'cell': 'NonscalingCell',  'block': 'DenseBlock',     'kernel': 5},
    'den_n_7': {'cell': 'NonscalingCell',  'block': 'DenseBlock',     'kernel': 7},
    'den_u_3': {'cell': 'UpscalingCell',   'block': 'DenseBlock',     'kernel': 3},
    'den_u_5': {'cell': 'UpscalingCell',   'block': 'DenseBlock',     'kernel': 5},
    'den_u_7': {'cell': 'UpscalingCell',   'block': 'DenseBlock',     'kernel': 7},
    'inc_d_3': {'cell': 'DownscalingCell', 'block': 'InceptionBlock', 'kernel': 3},
    'inc_d_5': {'cell': 'DownscalingCell', 'block': 'InceptionBlock', 'kernel': 5},
    'inc_d_7': {'cell': 'DownscalingCell', 'block': 'InceptionBlock', 'kernel': 7},
    'inc_n_3': {'cell': 'NonscalingCell',  'block': 'InceptionBlock', 'kernel': 3},
    'inc_n_5': {'cell': 'NonscalingCell',  'block': 'InceptionBlock', 'kernel': 5},
    'inc_n_7': {'cell': 'NonscalingCell',  'block': 'InceptionBlock', 'kernel': 7},
    'inc_u_3': {'cell': 'UpscalingCell',   'block': 'InceptionBlock', 'kernel': 3},
    'inc_u_5': {'cell': 'UpscalingCell',   'block': 'InceptionBlock', 'kernel': 5},
    'inc_u_7': {'cell': 'UpscalingCell',   'block': 'InceptionBlock', 'kernel': 7},
    'ide_d':   {'cell': 'DownscalingCell', 'block': 'IdentityBlock',             },
    'ide_n':   {'cell': 'NonscalingCell',  'block': 'IdentityBlock',             },   
    'ide_u':   {'cell': 'UpscalingCell',   'block': 'IdentityBlock',             },
    'res_d_3': {'cell': 'DownscalingCell', 'block': 'ResNetBlock',    'kernel': 3},
    'res_d_5': {'cell': 'DownscalingCell', 'block': 'ResNetBlock',    'kernel': 5},
    'res_d_7': {'cell': 'DownscalingCell', 'block': 'ResNetBlock',    'kernel': 7},
    'res_n_3': {'cell': 'NonscalingCell',  'block': 'ResNetBlock',    'kernel': 3},
    'res_n_5': {'cell': 'NonscalingCell',  'block': 'ResNetBlock',    'kernel': 5},
    'res_n_7': {'cell': 'NonscalingCell',  'block': 'ResNetBlock',    'kernel': 7},
    'res_u_3': {'cell': 'UpscalingCell',   'block': 'ResNetBlock',    'kernel': 3},
    'res_u_5': {'cell': 'UpscalingCell',   'block': 'ResNetBlock',    'kernel': 5},
    'res_u_7': {'cell': 'UpscalingCell',   'block': 'ResNetBlock',    'kernel': 7},
    'vgg_d_3': {'cell': 'DownscalingCell', 'block': 'VGGBlock',       'kernel': 3},
    'vgg_d_5': {'cell': 'DownscalingCell', 'block': 'VGGBlock',       'kernel': 5},
    'vgg_d_7': {'cell': 'DownscalingCell', 'block': 'VGGBlock',       'kernel': 7},
    'vgg_n_3': {'cell': 'NonscalingCell',  'block': 'VGGBlock',       'kernel': 3},
    'vgg_n_5': {'cell': 'NonscalingCell',  'block': 'VGGBlock',       'kernel': 5},
    'vgg_n_7': {'cell': 'NonscalingCell',  'block': 'VGGBlock',       'kernel': 7},
    'vgg_u_3': {'cell': 'UpscalingCell',   'block': 'VGGBlock',       'kernel': 3},
    'vgg_u_5': {'cell': 'UpscalingCell',   'block': 'VGGBlock',       'kernel': 5},
    'vgg_u_7': {'cell': 'UpscalingCell',   'block': 'VGGBlock',       'kernel': 7},
}

In [ ]:
patch_size = (128, 128)
batch_size = 32
num_classes = 2
stem_filters = 32
max_depth = 4

In [ ]:
model = build_net((*patch_size, 1), stem_filters, max_depth, num_classes, fn_dict, layer_list)#, cell_list)
#model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(
        model,
        to_file='model.png',
        show_shapes=True,
        show_layer_names=True,
        rankdir='TB'
)

Image('model.png')